# Importando as Bibliotecas

In [432]:
# import pandas
import pandas as pd
import seaborn as sn

In [433]:
import spacy

In [434]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from imblearn.over_sampling import SMOTE

In [435]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [436]:
import re
import pickle

# Conhecendo os dados

In [437]:
df = pd.read_csv(filepath_or_buffer="data/Dataset_5971.csv", sep=",")

In [438]:
display(df)

,LABEL,TEXT,URL,EMAIL,PHONE
0,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...,No,No,No
1,ham,What's up? Do you want me to come online? If y...,No,No,No
2,ham,So u workin overtime nigpun?,No,No,No
3,ham,"Also sir, i sent you an email about how to log...",No,No,No
4,Smishing,Please Stay At Home. To encourage the notion o...,No,No,No
...,...,...,...,...,...
5965,ham,Becoz its &lt;#&gt; jan whn al the post ofic...,No,No,No
5966,ham,Its a valentine game. . . send dis msg to all ...,No,No,No
5967,ham,We r outside already.,No,No,No
5968,ham,The Xmas story is peace.. The Xmas msg is love...,No,No,No


In [439]:
df.dtypes

LABEL    object
TEXT     object
URL      object
EMAIL    object
PHONE    object
dtype: object

In [440]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5970 entries, 0 to 5969
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LABEL   5970 non-null   object
 1   TEXT    5970 non-null   object
 2   URL     5970 non-null   object
 3   EMAIL   5970 non-null   object
 4   PHONE   5970 non-null   object
dtypes: object(5)
memory usage: 233.3+ KB


In [441]:
df.describe()

,LABEL,TEXT,URL,EMAIL,PHONE
count,5970,5970,5970,5970,5970
unique,5,5949,2,2,2
top,ham,The greatest test of courage on earth is to be...,No,No,No
freq,4842,2,5763,5951,5125


In [442]:
df["LABEL"].unique()

array(['ham', 'Smishing', 'spam', 'Spam', 'smishing'], dtype=object)

In [443]:
df["URL"].unique()

array(['No', 'yes'], dtype=object)

In [444]:
df["EMAIL"].unique()

array(['No', 'yes'], dtype=object)

In [445]:
df["PHONE"].unique()

array(['No', 'yes'], dtype=object)

In [446]:
len([x for x in df.duplicated() if x == True])

17

In [447]:
i = 0
index_dup = []
for x in df.duplicated():
    if x == True:
        index_dup.append(i)
    i += 1

In [448]:
df.iloc[index_dup]

,LABEL,TEXT,URL,EMAIL,PHONE
1350,Smishing,UR GOING 2 BAHAMAS! CallFREEFONE 08081560665 a...,No,No,yes
1802,Smishing,Someone U know has asked our dating service 2 ...,No,No,yes
2961,ham,He remains a bro amongst bros,No,No,No
3690,ham,Reverse is cheating. That is not mathematics.,No,No,No
3872,ham,The sign of maturity is not when we start sayi...,No,No,No
3906,Smishing,U have a secret admirer. REVEAL who thinks U R...,No,No,yes
3907,Smishing,Congrats! 2 mobile 3G Videophones R yours. cal...,No,No,yes
4140,ham,The greatest test of courage on earth is to be...,No,No,No
4729,ham,"Sorry,in meeting I'll call later",No,No,No
4925,Smishing,URGENT!! Your 4* Costa Del Sol Holiday or £500...,No,No,yes


In [449]:
df[df["TEXT"] == "Sorry,in meeting I'll call later"]

,LABEL,TEXT,URL,EMAIL,PHONE
415,ham,"Sorry,in meeting I'll call later",No,No,No
4729,ham,"Sorry,in meeting I'll call later",No,No,No


In [450]:
df[df["TEXT"] == "Arun can u transfr me d amt"]

,LABEL,TEXT,URL,EMAIL,PHONE
413,ham,Arun can u transfr me d amt,No,No,No
5341,ham,Arun can u transfr me d amt,No,No,No


# Transformando os dados

In [451]:
df_transformado = df.copy()

In [452]:
df_transformado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5970 entries, 0 to 5969
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LABEL   5970 non-null   object
 1   TEXT    5970 non-null   object
 2   URL     5970 non-null   object
 3   EMAIL   5970 non-null   object
 4   PHONE   5970 non-null   object
dtypes: object(5)
memory usage: 233.3+ KB


In [453]:
df_transformado["LABEL"].unique()

array(['ham', 'Smishing', 'spam', 'Spam', 'smishing'], dtype=object)

In [454]:
df_transformado["LABEL"].replace("Spam", "spam", inplace=True)
df_transformado["LABEL"].replace("Smishing", "smishing", inplace=True)
df_transformado["LABEL"].replace(["ham", "spam", "smishing"], [-1, 0, 1], inplace=True)

In [455]:
df_transformado["URL"].replace(["No", "yes"], [0, 1], inplace=True)
df_transformado["EMAIL"].replace(["No", "yes"], [0, 1], inplace=True)
df_transformado["PHONE"].replace(["No", "yes"], [0, 1], inplace=True)

In [456]:
display(df_transformado["URL"].unique())
display(df_transformado["EMAIL"].unique())
display(df_transformado["PHONE"].unique())

array([0, 1])

array([0, 1])

array([0, 1])

In [457]:
df_transformado.drop(["URL", "EMAIL", "PHONE"], axis=1, inplace=True)

In [458]:
len([x for x in df_transformado.duplicated() if x == True]) ## Ele aumenta por causa do tratamento que se fez para as palvras minusculas

20

In [459]:
df_transformado.drop_duplicates(inplace=True)

In [460]:
display(len(df_transformado))
display(len(df))

display(len(df) - len(df_transformado))

5950

5970

20

In [461]:
df_transformado.reset_index(drop=True, inplace=True)

In [462]:
display(df.head())
display(df_transformado.head())

,LABEL,TEXT,URL,EMAIL,PHONE
0,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...,No,No,No
1,ham,What's up? Do you want me to come online? If y...,No,No,No
2,ham,So u workin overtime nigpun?,No,No,No
3,ham,"Also sir, i sent you an email about how to log...",No,No,No
4,Smishing,Please Stay At Home. To encourage the notion o...,No,No,No


,LABEL,TEXT
0,-1,Your opinion about me? 1. Over 2. Jada 3. Kusr...
1,-1,What's up? Do you want me to come online? If y...
2,-1,So u workin overtime nigpun?
3,-1,"Also sir, i sent you an email about how to log..."
4,1,Please Stay At Home. To encourage the notion o...


# Analisando os dados

In [463]:
display(len(df_transformado[df_transformado["LABEL"] == -1]))
display(len(df_transformado[df_transformado["LABEL"] == 0]))
display(len(df_transformado[df_transformado["LABEL"] == 1]))

4832

489

629

# NLP

#### 1500 para treino e o resto para validação

In [464]:
#nlp = spacy.load("modelo\en_core_web_trf-3.7.1\en_core_web_trf\en_core_web_trf-3.7.1")
nlp = spacy.load("en_core_web_trf")

In [465]:
def colocar_palavras_em_minusculo(text):
    texto_lower = text.lower()
    return texto_lower

In [466]:
def remover_stopwords(text):
    doc = nlp(text)
    frases_sem_stopwords = " ".join([token.text for token in doc if not token.is_stop])
    return frases_sem_stopwords

In [467]:
def remover_espacos(text):
    frases_sem_espaco_frente = " ".join(text.split())
    return frases_sem_espaco_frente

In [468]:
def remover_caractere_especial(text):
    texto_sem_cs = re.sub(r"ï¿½", r"", text)
    texto_sem_cs = re.sub(r"&lt;([^>]+)&gt;", r"", texto_sem_cs)
    texto_sem_cs = re.sub(r"�", r" ", texto_sem_cs)
    texto_sem_cs = re.sub(r"ü", r"u", texto_sem_cs)
    texto_sem_cs = re.sub(r"Ü", r"U", texto_sem_cs)
    texto_sem_cs = re.sub(r"\(", r" ", texto_sem_cs)
    texto_sem_cs = re.sub(r"&amp;", r"", texto_sem_cs)
    return texto_sem_cs

In [469]:
def remover_emoji(text):
    text_sem_emoji = re.sub(r"((:|;)(-){0,1}(\)|\(|p))", f"", text)
    return text_sem_emoji

In [470]:
def remover_pontos(text):
    doc = nlp(text)
    frases_sem_ponto = " ".join([token.text for token in doc if not token.is_punct])
    return frases_sem_ponto

In [471]:
def lemmatizar(text):
    doc = nlp(text)
    frases_lematizadas = " ".join([token.lemma_ for token in doc])
    return frases_lematizadas

In [472]:
def substituir_numero_telefone(text):
    texto_com_numero_padronizado = re.sub(r"((\d{4}\s\d{3}\s\d{4})|(\d{10,14})|(\(\d{3}\)\d{3}-\d{4})|(\d{2}\w{9}))(\s|$|\/)", f" ph_number ", text)
    return texto_com_numero_padronizado

In [473]:
def substituir_email(text):
    texto_com_email_padronizado = re.sub(r"\S+@\S+($){0,1}", f" email ", text)
    return texto_com_email_padronizado

In [474]:
def pre_processamento(text):
    texto_processado = text
    print(texto_processado)
    texto_processado = colocar_palavras_em_minusculo(texto_processado)
    texto_processado = remover_emoji(texto_processado)
    texto_processado = remover_caractere_especial(texto_processado)
    texto_processado = remover_pontos(texto_processado)
    texto_processado = remover_stopwords(texto_processado)
    texto_processado = substituir_numero_telefone(texto_processado)
    texto_processado = substituir_email(texto_processado)
    texto_processado = remover_espacos(texto_processado)  
    texto_processado = lemmatizar(texto_processado)
    print(texto_processado)
    return texto_processado

In [475]:
df_transformado["PROCESSED_TEXT"] = df_transformado["TEXT"].apply(pre_processamento)

Your opinion about me? 1. Over 2. Jada 3. Kusruthi 4. Lovable 5. Silent 6. Spl character 7. Not matured 8. Stylish 9. Simple Pls reply..
opinion 1 2 jada 3 kusruthi 4 lovable 5 silent 6 spl character 7 matured 8 stylish 9 simple pl reply
What's up? Do you want me to come online? If you are free we can talk sometime�


want come online free talk
So u workin overtime nigpun?
u workin overtime nigpun
Also sir, i sent you an email about how to log into the usc payment portal. I.ll send you another message that should explain how things are back home. Have a great weekend.
sir send email log usc payment portal i.ll send message explain thing home great weekend
Please Stay At Home. To encourage the notion of staying at home. All tax-paying citizens are entitled to �305.96 or more emergency refund. smsg.io/fCVbD
stay home encourage notion stay home tax pay citizen entitle 305.96 emergency refund smsg.io/fcvbd
BankOfAmerica Alert 137943. Please follow http://bit.do/cgjK-and re-activate
bankofamerica alert 137943 follow http://bit.do/cgjk-and activate
Sorry dude. Dont know how i forgot. Even after Dan reminded me. Sorry. Hope you guys had fun.
sorry dude nt know forget dan remind sorry hope guy fun
I don't quite know what to do. I still can't get hold of anyone. I cud pick you up bout 7.30pm and we can see i

In [476]:
i = 0
for x in df_transformado["PROCESSED_TEXT"]:
    if re.search(r"\d{4}\w{7}", x) is not None:
        print(df_transformado["LABEL"].iloc[i])
        print(df_transformado["TEXT"].iloc[i])
        print(df_transformado["PROCESSED_TEXT"].iloc[i])
    i += 1

1
URGENT! You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18
urgent win 1 week free membership £ 100,000 prize jackpot txt word claim 81010 t&c www.dbuk.net lccltd pobox 4403ldnw1a7rw18
1
	FREE MESSAGE Activate your 500 FREE Text Messages by replying to this message with the word FEE, visit www.07781482378.com
free message activate 500 free text message reply message word fee visit www.07781482378.com
1
FREE MESSAGE Activate your 500 FREE Text Messages by replying to this message with the word FREE For terms & conditions, visit www.07781482378.com
free message activate 500 free text message reply message word free term condition visit www.07781482378.com
0
	http//tms. widelive.com/index. wml?id=820554ad0a1705572711&first=true¡C C Ringtone¡
http//tms widelive.com/index wml?id=820554ad0a1705572711&first = true¡c c ringtone
1
	FREE MESSAGE Activate your 500 FREE Text Messages by replying to th

In [477]:
i = 0
for x in df_transformado["PROCESSED_TEXT"].duplicated():
    if x == True:
        print([df_transformado["LABEL"].iloc[i]])
        print(i, [df_transformado["PROCESSED_TEXT"].iloc[i]])
    i += 1

[-1]
136 ['nt want hear']
[1]
224 ['ph_number immediately urgent message waiting']
[1]
319 ['want new video handset 750 time network min unlimited text camcorder reply ph_number del sat']
[-1]
417 ['wat r u']
[1]
430 ['ph_number immediately urgent message waiting']
[1]
500 ['dear v0daf0ne customer 5,00,000 winner guarantee caller prize final attempt contact claim ph_number']
[1]
548 ['ph_number immediately urgent message waiting']
[1]
708 ['u secret admirer look 2 contact u find r*reveal think ur special ph_number']
[-1]
729 ['']
[-1]
785 ['gud mrng dear hav nice day']
[1]
824 ['dear v0daf0ne customer 5,00,000 winner guarantee caller prize final attempt contact claim ph_number']
[1]
868 ['ph_number urgent mobile ph_number win £ 2,000 bonus caller prize 02/06/03 2nd attempt reach ph_number asap']
[-1]
902 ['']
[0]
927 ['free ringtone -sub weekly ringtone 1st week free send subpoly 81618-?3 week stop sms- ph_number']
[1]
989 ['free message activate 500 free text message reply message wor

In [478]:
i = 0
for x in df_transformado["PROCESSED_TEXT"] == "":
    if x == True:
        print([df_transformado["LABEL"].iloc[i]])
        print(i, [df_transformado["PROCESSED_TEXT"].iloc[i]])
    i += 1

[-1]
543 ['']
[-1]
729 ['']
[-1]
902 ['']
[-1]
2095 ['']
[-1]
2784 ['']
[-1]
3111 ['']
[-1]
4126 ['']
[-1]
4471 ['']
[-1]
4771 ['']
[-1]
4954 ['']
[-1]
5270 ['']
[-1]
5470 ['']
[-1]
5510 ['']
[-1]
5945 ['']


In [479]:
df_transformado.drop_duplicates(subset=["LABEL","PROCESSED_TEXT"], inplace=True)

In [480]:
#Remover os que estão duplicados
#TODO: conferir se foram os melhores dados
# df_transformado.drop_duplicates(subset=["PROCESSED_TEXT"], inplace=True)

In [481]:
df_transformado.head(20)

,LABEL,TEXT,PROCESSED_TEXT
0,-1,Your opinion about me? 1. Over 2. Jada 3. Kusr...,opinion 1 2 jada 3 kusruthi 4 lovable 5 silent...
1,-1,What's up? Do you want me to come online? If y...,want come online free talk
2,-1,So u workin overtime nigpun?,u workin overtime nigpun
3,-1,"Also sir, i sent you an email about how to log...",sir send email log usc payment portal i.ll sen...
4,1,Please Stay At Home. To encourage the notion o...,stay home encourage notion stay home tax pay c...
5,1,BankOfAmerica Alert 137943. Please follow http...,bankofamerica alert 137943 follow http://bit.d...
6,-1,Sorry dude. Dont know how i forgot. Even after...,sorry dude nt know forget dan remind sorry hop...
7,-1,I don't quite know what to do. I still can't g...,know hold cud pick bout 7.30pm pub
8,-1,Ok lor. Anyway i thk we cant get tickets now c...,ok lor thk nt ticket cos like late u wan 2 loo...
9,-1,Wat r u doing now?,wat r u


In [482]:
# TODO: Melhorar alguns lemmas, colocar stemm, retirar os caracteres especiais
display(df_transformado["PROCESSED_TEXT"].iloc[1])
display(df_transformado["TEXT"].iloc[1])

'want come online free talk'

"What's up? Do you want me to come online? If you are free we can talk sometime�"

In [483]:
display(len(df_transformado[df_transformado["LABEL"] == -1]))
display(len(df_transformado[df_transformado["LABEL"] == 0]))
display(len(df_transformado[df_transformado["LABEL"] == 1]))

4769

469

487

# Treinamento

In [484]:
df_teste = df_transformado.iloc[:1500].copy()
df_validacao = df_transformado.iloc[1500:].copy()

In [485]:
x = df_teste["PROCESSED_TEXT"]
y = df_teste["LABEL"]

In [486]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [487]:
class_weights = {-1: 1, 0: 2, 1: 4}

In [488]:
#TODO: Precisa levar esse cara para lá
vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

In [489]:
vectorizer.transform(["BankOfAmerica Alert 137943. Please follow http://bit.do/cgjK-and re-activate"]).toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

##### Logistic Regression

In [490]:
model = LogisticRegression(solver="lbfgs", multi_class="multinomial", random_state=42, class_weight=class_weights)
model.fit(x_train_tfidf, y_train)

LogisticRegression(class_weight={-1: 1, 0: 2, 1: 4}, multi_class='multinomial',
                   random_state=42)

In [491]:
y_pred = model.predict(x_test_tfidf)

In [492]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print(f'Relatório de Classificação:\n{report}')

Acurácia do modelo: 0.91
Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.95      0.99      0.97       371
           0       0.64      0.22      0.33        41
           1       0.70      0.87      0.78        38

    accuracy                           0.91       450
   macro avg       0.76      0.69      0.69       450
weighted avg       0.90      0.91      0.90       450



#### Random Florest

In [493]:
rf_classifier = RandomForestClassifier(random_state=42, class_weight=class_weights)
rf_classifier.fit(x_train_tfidf, y_train)

RandomForestClassifier(class_weight={-1: 1, 0: 2, 1: 4}, random_state=42)

In [494]:
y_pred = rf_classifier.predict(x_test_tfidf)

In [495]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print(f'Relatório de Classificação:\n{report}')

Acurácia do modelo: 0.92
Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.96      1.00      0.98       371
           0       0.80      0.29      0.43        41
           1       0.69      0.87      0.77        38

    accuracy                           0.92       450
   macro avg       0.82      0.72      0.72       450
weighted avg       0.92      0.92      0.91       450



#### SVM

In [496]:
svm_classifier = SVC(kernel='linear',random_state=42, class_weight=class_weights)
svm_classifier.fit(x_train_tfidf, y_train)

SVC(class_weight={-1: 1, 0: 2, 1: 4}, kernel='linear', random_state=42)

In [497]:
y_pred = svm_classifier.predict(x_test_tfidf)

In [498]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print(f'Relatório de Classificação:\n{report}')

Acurácia do modelo: 0.92
Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.96      0.99      0.97       371
           0       0.64      0.39      0.48        41
           1       0.73      0.79      0.76        38

    accuracy                           0.92       450
   macro avg       0.78      0.72      0.74       450
weighted avg       0.91      0.92      0.91       450



# Validação

In [499]:
x_v = vectorizer.transform(df_validacao["PROCESSED_TEXT"])
y_v = df_validacao["LABEL"]
y_pred = model.predict(x_v)

In [500]:
x_v = vectorizer.transform(df_validacao["PROCESSED_TEXT"])
y_v = df_validacao["LABEL"]
y_pred = rf_classifier.predict(x_v)

In [501]:
x_v = vectorizer.transform(df_validacao["PROCESSED_TEXT"])
y_v = df_validacao["LABEL"]
y_pred = svm_classifier.predict(x_v)

In [502]:
accuracy = accuracy_score(y_v, y_pred)
report = classification_report(y_v, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print(f'Relatório de Classificação:\n{report}')

Acurácia do modelo: 0.94
Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.97      1.00      0.98      3551
           0       0.77      0.48      0.59       345
           1       0.75      0.84      0.79       329

    accuracy                           0.94      4225
   macro avg       0.83      0.77      0.79      4225
weighted avg       0.94      0.94      0.94      4225



In [503]:
with open("modelo/modelo.pkl", 'wb') as arquivo:
    pickle.dump(model, arquivo)

with open("modelo/vectorize.pkl", "wb") as arquivo:
    pickle.dump(vectorizer, arquivo)

# Teste de exportação

In [504]:
test = vectorizer.transform(["bankofamerica alert 137943 follow http://bit.do/cgjk-and activate"])

In [505]:
test.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [506]:
pred = model.predict(test)

In [507]:
pred

array([1])

In [508]:
with open('modelo/modelo.pkl', 'rb') as arquivo:
    modelo_test = pickle.load(arquivo)

with open('modelo/vectorize.pkl', 'rb') as arquivo:
    vectorize = pickle.load(arquivo)

In [509]:
vectorize

TfidfVectorizer()

In [510]:
modelo_test

LogisticRegression(class_weight={-1: 1, 0: 2, 1: 4}, multi_class='multinomial',
                   random_state=42)

In [511]:
val = vectorize.transform(["bankofamerica alert 137943 follow http://bit.do/cgjk-and activate"])

In [512]:
val.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [513]:
pred_val = modelo_test.predict(val)

In [514]:
pred_val

array([1])